In [1]:

import requests
import json
import pandas as pd
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

# Complexes data Extraction

In [3]:
url = f"https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key={API_KEY}"
resp = requests.get(url = url)

In [4]:

data = resp.json()
with open("../data/json_files/complex_data.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)

In [5]:
complexes = []
venues = []
for comp in data.get('complexes', []):
    id = comp.get('id')
    complexes.append({'complex_id': id, 'name': comp.get('name')})
    for venue in comp.get('venues', []):
        venues.append({'id' : venue.get('id'), 'name': venue.get('name'), 'complex_id': id, 'city_name': venue.get('city_name'), 
                     'country_name': venue.get('country_name'), 'country_code' : venue.get('country_code'), 'timezone' : venue.get('timezone')
                     })

In [6]:
complex_df = pd.DataFrame(complexes)
venues_df = pd.DataFrame(venues)

In [7]:
complex_df.to_csv('../data/raw_file/complex.csv', index = False)
venues_df.to_csv('../data/raw_file/venues.csv', index = False)

# Ranking Data Extraction

In [8]:

url = f'https://api.sportradar.com/tennis/trial/v3/en/rankings.json?api_key={API_KEY}'
response = requests.get(url)
data = response.json()

In [9]:
with open('../data/json_files/ranking.json', 'w') as f:
    json.dump(data, f, indent = 2)


In [10]:
Competitor_Rankings = []
Competitors_list = []

In [11]:
for ranking in data.get('rankings', []):
    gender = ranking.get('gender')
    tour = ranking.get('name')
    year = ranking.get('year')
    week = ranking.get('week')
    for x in ranking.get('competitor_rankings', []):
        competitors = x.get('competitor')
        Competitor_Rankings.append({'rank': x.get('rank'), 'movement': x.get('movement'), 'points': x.get('points'), 'competitions_played': x.get('competitions_played'), 'competitor_id': competitors.get('id'), 'gender': gender, 'year': year, 'week': week, 'tour': tour})
        Competitors_list.append({'id': competitors.get('id'),'name': competitors.get('name'), 'country': competitors.get('country'), 'country_code': competitors.get('country_code'), 'abbreviation': competitors.get('abbreviation')})

In [12]:
Competitor_Rankings_df = pd.DataFrame(Competitor_Rankings)
competitor_df = pd.DataFrame(Competitors_list)

In [13]:
Competitor_Rankings_df.to_csv('../data/raw_file/Competitor_Rankings.csv', index = False)
competitor_df.to_csv('../data/raw_file/competitor.csv', index = False)

# Categories Table

In [14]:

url = f'https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key={API_KEY}'
response = requests.get(url)


In [15]:
data = response.json()

with open('../data/json_files/competitions.json', 'w') as f:
    json.dump(data, f, indent = 2)

In [16]:
categories = []
competitions = []

for comp in data.get('competitions', []):
    category_parse = comp.get('category')
    categories.append({'id': category_parse.get('id'), 'name': category_parse.get('name')})
    competitions.append({'id': comp.get('id'), 'name': comp.get('name'), 'parent_id': comp.get('parent_id'), 
                          'type': comp.get('type'), 'gender': comp.get('gender'), 'category_id': category_parse.get('id')})

In [17]:
category_df = pd.DataFrame(categories)
competition_df = pd.DataFrame(competitions)

In [18]:
category_df.to_csv('../data/raw_file/category.csv', index = False)
competition_df.to_csv('../data/raw_file/competition.csv', index = False)